In [1]:
import deepSI
import numpy as np
from casadi import *
from NonlinearController.model_utils import *
import time
import torch
from functorch import jacrev, jacfwd, vmap

In [12]:
def lambda_trap(x0,u0,nx,nu,ny,Jfx,Jfu,Jhx,stages):
    # FUNCTION LAMBDA_SIMPSON
    # Simpson rule integrator between 0 and 1 with chosen resolution (stages)
    # used to get A,B matrices symbolically to be used at gridpoints
    
    A = np.zeros([nx,nx])
    B = np.zeros([nx,nu])
    C = np.zeros([ny,nx])
    lambda0 = 0
    dlam = 1/stages

    x = lambda lam: lam*x0
    u = lambda lam: lam*u0

    for i in np.arange(stages):
        A = A + dlam*1/2*(Jfx(x(lambda0), u(lambda0)) + Jfx(x(lambda0+dlam), u(lambda0+dlam)))
        B = B + dlam*1/2*(Jfu(x(lambda0), u(lambda0)) + Jfu(x(lambda0+dlam), u(lambda0+dlam)))
        C = C + dlam*1/2*(Jhx(x(lambda0), u(lambda0)) + Jhx(x(lambda0+dlam), u(lambda0+dlam)))
        lambda0 = lambda0 + dlam
            
    return A,B,C

def lambda_simpson(x,u,nx,nu,ny,Jfx,Jfu,Jhx,stages):
    # FUNCTION LAMBDA_SIMPSON
    # Simpson rule integrator between 0 and 1 with chosen resolution (stages)
    # used to get A,B matrices symbolically to be used at gridpoints
    
    A = np.zeros([nx,nx])
    B = np.zeros([nx,nu])
    C = np.zeros([ny,nx])
    lambda0 = 0
    dlam = 1/stages

    for i in np.arange(stages):
        A = A + dlam*1/6*(Jfx(lambda0*x,lambda0*u) + 4*Jfx((lambda0+dlam/2)*x,(lambda0+dlam/2)*u) + Jfx((lambda0+dlam)*x,(lambda0+dlam)*u))
        B = B + dlam*1/6*(Jfu(lambda0*x,lambda0*u) + 4*Jfu((lambda0+dlam/2)*x,(lambda0+dlam/2)*u) + Jfu((lambda0+dlam)*x,(lambda0+dlam)*u))
        C = C + dlam*1/6*(Jhx(lambda0*x,lambda0*u) + 4*Jhx((lambda0+dlam/2)*x,(lambda0+dlam/2)*u) + Jhx((lambda0+dlam)*x,(lambda0+dlam)*u))
        lambda0 = lambda0 + dlam
            
    return A,B,C


In [59]:
model = deepSI.load_system("NonlinearController/trained_models/unbalanced/ObserverUnbalancedDisk_dt01_nab_4_SNR_30_e250")
n_stages = 20
Nc = 10
n_int_comp = 3
threads = 8

# declared sym variables
nx = model.nx
x = MX.sym("x",nx,1)
nu = model.nu if model.nu is not None else 1
u = MX.sym("u",nu,1)
ny = model.ny if model.ny is not None else 1

# convert torch nn to casadi function
x_rhs = CasADi_Fn(model, x, u)
f = Function('f', [x, u], [x_rhs])
y_rhs = CasADi_Hn(model, x)
h = Function('h', [x], [y_rhs])

correction_f = f(np.zeros((nx,1)), 0)
x_rhs_c = x_rhs - correction_f
correction_h = h(np.zeros((nx,1)))
y_rhs_c = y_rhs - correction_h

Jfx = Function("Jfx", [x, u], [jacobian(x_rhs_c,x)])
Jfu = Function("Jfu", [x, u], [jacobian(x_rhs_c,u)])
Jhx = Function("Jhx", [x, u], [jacobian(y_rhs_c,x)])

x0 = MX.sym("x0",nx,1)
u0 = MX.sym("u0",nu,1)

[A_sym, B_sym, C_sym] = lambda_simpson(x0,u0,nx,nu,ny,Jfx,Jfu,Jhx,n_stages)
lpv_A = Function("get_A",[x0,u0],[A_sym])
lpv_B = Function("get_B",[x0,u0],[B_sym])
lpv_C = Function("get_C",[x0,u0],[C_sym])

lpv_A_Nc = lpv_A.map(Nc, "thread", threads)
lpv_B_Nc = lpv_B.map(Nc, "thread", threads)
lpv_C_Nc = lpv_C.map(Nc, "thread", threads)

In [60]:
Lambda = np.array([])
lambda0 = 0
dlam = 1/n_stages

for i in np.arange(n_stages):
    Lambda = np.hstack((Lambda, lambda0, lambda0 + dlam/2, lambda0 + dlam)) # Simpson
    # Lambda = np.hstack((Lambda, lambda0, lambda0 + dlam)) # Trapezium
    lambda0 = lambda0 + dlam

In [61]:
lpv_A_lam = Jfx.map(Nc*n_stages*n_int_comp, "thread", threads)
lpv_B_lam = Jfu.map(Nc*n_stages*n_int_comp, "thread", threads)
lpv_C_lam = Jhx.map(Nc*n_stages*n_int_comp, "thread", threads)

In [62]:
if torch.cuda.is_available(): 
    dev = "cuda:0" 
else: 
    dev = "cpu" 
device = torch.device(dev) 
JacF = torch.vmap(torch.func.jacrev(model.fn.to(device).float(), argnums=(0,1)))
JacH = torch.vmap(torch.func.jacrev(model.hn.to(device).float()))

In [64]:
n_iter = 1000
torch.set_num_threads(threads)

func_end_time = 0.0
cas_end_time = 0.0
stages_end_time = 0.0

for i in range(n_iter):
    X = np.random.rand(nx,Nc)
    U = np.random.rand(nu,Nc)

    Xlam = np.kron(X, Lambda)
    Ulam = np.kron(U, Lambda)

    batch_size = Nc*n_stages*n_int_comp

    start_time = time.time()

    x = torch.reshape(torch.tensor(Xlam[np.newaxis].T, device=device),(batch_size,1,2)).float()
    u = torch.reshape(torch.tensor(Ulam[np.newaxis].T, device=device),(batch_size,1,1)).float()

    fA, fB = JacF(x,u)
    fC = JacH(x)

    temp1 = fA.cpu()
    temp2 = fB.cpu()
    temp3 = fC.cpu()

    func_end_time += time.time() - start_time

    # start_time = time.time()

    # pA = lpv_A_Nc(X,U)
    # pB = lpv_B_Nc(X,U)
    # pC = lpv_C_Nc(X,U)

    # cas_end_time += time.time() - start_time

    start_time = time.time()

    tA = lpv_A_lam(Xlam,Ulam)
    tB = lpv_B_lam(Xlam,Ulam)
    tC = lpv_C_lam(Xlam,Ulam)

    stages_end_time += time.time() - start_time

print("functorch: " + str(func_end_time))
print("CasADi Nc: " + str(cas_end_time))
print("CasADi stages: " + str(stages_end_time))

functorch: 5.178942680358887
CasADi Nc: 0.0
CasADi stages: 14.02291464805603


In [8]:
pA = lpv_A_Nc(X,U)
pB = lpv_B_Nc(X,U)
pC = lpv_C_Nc(X,U)

In [9]:
n_iter = 1

list_A = np.zeros([Nc*nx, nx])
list_B = np.zeros([Nc*nx, nu])
list_C = np.zeros([Nc*ny, nx])

mult_A = np.stack((np.ones((nx,nx)), np.ones((nx,nx))*4, np.ones((nx,nx))))
mult_B = np.stack((np.ones((nx,nu)), np.ones((nx,nu))*4, np.ones((nx,nu))))
mult_C = np.vstack((np.ones((ny,nx)), np.ones((ny,nx))*4, np.ones((ny,nx))))

start_time = time.time()
for t in range(n_iter):

    for j in range(Nc):
        A = np.zeros((nx,nx))
        B = np.zeros([nx,nu])
        C = np.zeros((ny,nx))

        lambda0 = 0

        for i in range(n_stages):
            an = fA[n_int_comp*n_stages*j+(i)*n_int_comp:n_int_comp*n_stages*j+(i+1)*n_int_comp,0,:,0,:].detach().cpu().numpy()
            A = A + dlam*1/6*np.sum(np.multiply(mult_A, an), axis=0)

            bn = fB[n_int_comp*n_stages*j+(i)*n_int_comp:n_int_comp*n_stages*j+(i+1)*n_int_comp,0,:,0,:].detach().cpu().numpy()
            B = B + dlam*1/6*np.sum(np.multiply(mult_B, bn), axis=0)

            cn = fC[n_int_comp*n_stages*j+(i)*n_int_comp:n_int_comp*n_stages*j+(i+1)*n_int_comp,0,0,:].detach().cpu().numpy()
            C = C + dlam*1/6*np.sum(np.multiply(mult_C, cn), axis=0)[np.newaxis]

            lambda0 = lambda0 + dlam

        list_A[nx*(j):nx*(j+1),:] = A.copy()
        list_B[nx*(j):nx*(j+1),:] = B.copy()
        list_C[ny*(j):ny*(j+1),:] = C.copy()

end_time = time.time()
print("For Loops: " + str(end_time - start_time))


length_jac = n_stages*n_int_comp*Nc

mult_fA = torch.from_numpy(np.tile(np.vstack((np.ones((1,nx,nx)), 4*np.ones((1,nx,nx)), np.ones((1,nx,nx)))),(n_stages*Nc,1,1))).cuda()*dlam/6
mult_fB = torch.from_numpy(np.tile(np.vstack((np.ones((1,nx,nu)), 4*np.ones((1,nx,nu)), np.ones((1,nx,nu)))),(n_stages*Nc,1,1))).cuda()*dlam/6
mult_fC = torch.from_numpy(np.tile(np.vstack((np.ones((1,ny,nx)), 4*np.ones((1,ny,nx)), np.ones((1,ny,nx)))),(n_stages*Nc,1,1))).cuda()*dlam/6

list_fA = np.zeros([Nc*nx, nx])
list_fB = np.zeros([Nc*nx, nu])
list_fC = np.zeros([Nc*ny, nx])

start_time = time.time()

for t in range(n_iter):
    tempA = torch.tensor_split(torch.mul(fA.view((length_jac,nx,nx)), mult_fA).detach(), Nc)
    tempB = torch.tensor_split(torch.mul(fB.view((length_jac,nx,nu)), mult_fB).detach(), Nc)
    tempC = torch.tensor_split(torch.mul(fC.view((length_jac,ny,nx)), mult_fC).detach(), Nc)
    for i in range(Nc):
        list_fA[nx*(i):nx*(i+1),:] = torch.sum(tempA[i], axis=0).cpu().numpy()
        list_fB[nx*(i):nx*(i+1),:] = torch.sum(tempB[i], axis=0).cpu().numpy()
        list_fC[ny*(i):ny*(i+1),:] = torch.sum(tempC[i], axis=0).cpu().numpy()

end_time = time.time()
print("Torch short: " + str(end_time - start_time))

For Loops: 1.053025245666504
Torch short: 0.03500032424926758


In [10]:
np.allclose(list_A, list_fA), np.allclose(list_B, list_fB), np.allclose(list_C, list_fC)

(True, True, True)

In [11]:
list_A_1 = np.zeros([Nc*nx, nx])
list_B_1 = np.zeros([Nc*nx, nx])
list_C_1 = np.zeros([Nc*ny, nx])
for i in range(Nc):
    list_A_1[(nx*i):(nx*i+nx),:] = pA[:,i*nx:(i+1)*nx]
for i in range(Nc):
    list_B_1[(nx*i):(nx*i+nx),:] = pB[:,i*nu:(i+1)*nu]
for i in range(Nc):
    list_C_1[(ny*i):(ny*i+ny),:] = pC[:,i*nx:(i+1)*nx]

In [12]:
np.allclose(list_A_1, list_A), np.allclose(list_B_1, list_B), np.allclose(list_C_1, list_C)

(True, True, True)